In [1]:
from qtensor import Info, MPS, Gates, CircuitCX
import torch
import numpy as np
import matplotlib.pyplot as plt
import time

N = 5

info = Info()
mps = MPS(info)
mps.all_zeros_state(N)

gates = Gates(info)

gates_stochastic = [gates.Rn_random() for i in range(N)]

for i in range(N):
    mps.one_qubit_gate(gates_stochastic[i], i)
    
mps.two_qubit_gate(gates.CX(), 1)
mps.two_qubit_gate(gates.CX(), 2)
mps.two_qubit_gate(gates.CX(), 3)

u = gates.Rn_random()

rho_0 = mps.get_density_matrix(4)

rho_1 = np.dot(np.array(u, dtype=complex), np.dot(np.array(rho_0, dtype=complex),
                                                  np.array(u, dtype=complex).T.conjugate()))

mps.one_qubit_gate(u, 4)
rho_2 = np.array(mps.get_density_matrix(4))

print(mps.r)
print(np.sum(np.abs(rho_1 - rho_2) ** 2))

[1, 1, 2, 2, 2, 1]
9.704881360388806e-33


In [2]:
class MPO(object):
    def __init__(self, info):
        self.tt_cores = []
        self.info = info
        self.N = None
        self.r = []
        self.phys_ind = []

    def all_zeros_state(self, n):
        self.tt_cores = []
        self.r = [1]
        self.phys_ind = []
        for i in range(n):
            self.tt_cores.append(torch.reshape(torch.tensor([[1, 0], [0, 0]], dtype=self.info.data_type, device=self.info.device),
                                               (1, 2, 2, 1)))
            self.r.append(1)
            self.phys_ind.append((2, 2))
        self.N = n
        
    def get_element(self, list_of_index_1, list_of_index_2):
        matrix_list = [self.tt_cores[i][:, index1, index2, :] for i, (index1, index2) in enumerate(zip(list_of_index_1, list_of_index_2))]
        element = matrix_list[0]
        for matrix in matrix_list[1:]:
            element = torch.tensordot(element, matrix, dims=([1], [0]))
        return element[0][0]
    
    def construct(self, mps):
        self.N = mps.N
        
        self.r = []
        for i in range(self.N + 1):
            self.r.append(mps.r[i] * mps.r[i])
        self.phys_ind = []
        for i in range(self.N):
            self.phys_ind.append((mps.phys_ind[i], mps.phys_ind[i]))
        
        self.tt_cores = []
        for i in range(self.N):
            core1 = torch.conj(mps.tt_cores[i])
            core2 = mps.tt_cores[i]
            self.tt_cores.append(torch.reshape(torch.kron(core1, core2),
                                               (self.r[i], self.phys_ind[i][0], self.phys_ind[i][1], self.r[i + 1])))
            
    def diagonal(self):
        mps = MPS(self.info)
        mps.N = self.N
        mps.r = self.r
        mps.phys_ind = []
        for i in range(self.N):
            assert self.phys_ind[i][0] == self.phys_ind[i][1]
            mps.phys_ind.append(self.phys_ind[i][0])
        mps.tt_cores = []
        for i in range(self.N):
            mps.tt_cores.append(torch.zeros((mps.r[i], mps.phys_ind[i], mps.r[i+1]), dtype=self.info.data_type, device=self.info.device))
            for j in range(mps.phys_ind[i]):
                mps.tt_cores[i][:, j, :] = self.tt_cores[i][:, j, j, :]
        return mps
    
    def trace(self):
        return self.diagonal().get_sum()

In [3]:
mpo = MPO(info)

mpo.all_zeros_state(N)

print(mpo.get_element([0, 0, 0, 0, 1], [0, 0, 0, 0, 0]))

tensor(0.+0.j, dtype=torch.complex128)


In [4]:
mpo.construct(mps)

In [5]:
indices = []

def gen(pref):
    if len(pref) == N:
        global indices
        indices.append(pref)
        return
    for i in range(2):
        gen(pref + [i])
        
gen([])
for i1 in indices:
    for i2 in indices:
        val1 = mps.get_element(i1)
        val2 = mps.get_element(i2)
        corr = torch.conj(val1) * val2
        our = mpo.get_element(i1, i2)
        diff = torch.abs(corr - our)
        assert diff.item() < 1e-12

In [6]:
mpo.trace()

tensor(1.0000+0.j, dtype=torch.complex128)